<a href="https://colab.research.google.com/github/YangHyunu/SKT-FLY-AI-5/blob/main/4%EC%A3%BC%EC%B0%A8/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98/%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
import time
import copy
import glob
import shutil
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


In [ ]:
data_path = '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/train'

### 예제에서 사용할 이미지 데이터에 대한 전처리 방법을 정의

In [ ]:
transform = transforms.Compose(
                [
                    transforms.Resize([256, 256]), # 데이터 리사이징
                    transforms.RandomResizedCrop(224), # 랜덤하게 이미지를 자름 데이터 증강효과
                    transforms.RandomHorizontalFlip(), # 데이터 증강
                    transforms.ToTensor(),
                ])
train_dataset = torchvision.datasets.ImageFolder(
    data_path,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=8, # 너무 많은 하위 프로세스 방지
    shuffle=True
)

print(len(train_dataset))

385


### Resnet
- 사전 학습된 가중치 사용

In [ ]:
resnet18 = models.resnet18()

In [ ]:
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = True

set_parameter_requires_grad(resnet18)

### 완전 연결층

In [ ]:
resnet18.fc = nn.Linear(512,2)

In [ ]:
for name, param in resnet18.named_parameters():
  if param.requires_grad:
    print(name, param.data)

conv1.weight tensor([[[[ 4.0848e-02,  6.3158e-03, -4.6463e-02,  ...,  1.4430e-02,
            2.7523e-02, -1.3985e-02],
          [-1.5436e-02, -8.8933e-03,  1.9853e-02,  ..., -5.8091e-02,
           -4.0287e-02,  2.5934e-03],
          [ 6.4538e-03,  3.2023e-02, -3.1898e-02,  ..., -3.9315e-02,
            8.4643e-03, -2.0995e-02],
          ...,
          [ 2.7750e-02, -2.9782e-02, -1.9395e-03,  ..., -2.6083e-02,
            1.4164e-02,  5.7035e-02],
          [ 1.4206e-02,  4.3344e-02,  4.1206e-02,  ..., -4.0196e-02,
           -4.6990e-02,  3.7824e-02],
          [-1.7170e-02, -1.5598e-03,  1.6551e-02,  ...,  2.5672e-02,
           -2.0782e-03,  3.1642e-02]],

         [[-2.1365e-02,  1.1570e-02,  2.3851e-02,  ..., -1.7743e-02,
           -8.9848e-03,  4.7657e-03],
          [-5.8871e-03,  1.0941e-02, -1.6691e-04,  ..., -2.3334e-02,
           -2.1134e-02, -8.8843e-03],
          [ 1.5828e-02,  4.8460e-02,  7.6624e-03,  ...,  1.4882e-02,
            2.6702e-02,  2.3118e-02],
       

In [ ]:
model = models.resnet18()

for param in model.parameters():
    param.requires_grad = True

model.fc = torch.nn.Linear(512, 2)
for param in model.fc.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.fc.parameters())
cost = torch.nn.CrossEntropyLoss()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=13, is_train=True):
    since = time.time()
    acc_history = []
    loss_history = []
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            model.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)
        torch.save(model.state_dict(), os.path.join('/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/', '{0:0=2d}.pth'.format(epoch)))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))
    return acc_history, loss_history

In [ ]:
params_to_update = []
for name,param in resnet18.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

optimizer = optim.Adam(params_to_update)

	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.bn2.weight
	 layer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
train_acc_hist, train_loss_hist = train_model(resnet18, train_loader, criterion, optimizer, device)

Epoch 0/12
----------
Loss: 1.1095 Acc: 0.5299

Epoch 1/12
----------
Loss: 0.7595 Acc: 0.5299

Epoch 2/12
----------
Loss: 0.7418 Acc: 0.5403

Epoch 3/12
----------
Loss: 0.6734 Acc: 0.5896

Epoch 4/12
----------
Loss: 0.6880 Acc: 0.5896

Epoch 5/12
----------
Loss: 0.6776 Acc: 0.5610

Epoch 6/12
----------
Loss: 0.7210 Acc: 0.5792

Epoch 7/12
----------
Loss: 0.6897 Acc: 0.5714

Epoch 8/12
----------
Loss: 0.6527 Acc: 0.6234

Epoch 9/12
----------
Loss: 0.6709 Acc: 0.5688

Epoch 10/12
----------
Loss: 0.7120 Acc: 0.5506

Epoch 11/12
----------
Loss: 0.6563 Acc: 0.6000

Epoch 12/12
----------
Loss: 0.6457 Acc: 0.6468

Training complete in 1m 4s
Best Acc: 0.646753


### 테스트 데이터를 불러와 전처리함

In [ ]:
test_path = '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/test'

In [ ]:
transform = transforms.Compose(
                [
                    transforms.Resize(224), # 데이터 리사이징
                    transforms.CenterCrop(224), # 랜덤하게 이미지를 자름 데이터 증강효과
                    transforms.ToTensor(),
                ])
test_dataset = torchvision.datasets.ImageFolder(
    root=test_path,
    transform = transform
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    num_workers=1, # 너무 많은 하위 프로세스 방지
    shuffle=True
)

In [ ]:
print(len(test_dataset))

98


In [ ]:
def eval_model(model, dataloaders, device):
    since = time.time()
    acc_history = []
    best_acc = 0.0

    saved_models = glob.glob('/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/*.pth')
    saved_models.sort()
    print('saved_model', saved_models)

    for model_path in saved_models:
        print('Loading model', model_path)

        model.load_state_dict(torch.load(model_path))
        model.eval()
        model.to(device)
        running_corrects = 0

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(inputs)

            _, preds = torch.max(outputs.data, 1)
            preds[preds >= 0.5] = 1
            preds[preds < 0.5] = 0
            running_corrects += preds.eq(labels).int().sum()

        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        print('Acc: {:.4f}'.format(epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc.item())
        print()

    time_elapsed = time.time() - since
    print('Validation complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))

    return acc_history

In [ ]:
val_acc_hist = eval_model(resnet18, test_loader, device)

saved_model ['/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/00.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/01.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/02.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/03.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/04.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/05.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/06.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/07.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/08.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/09.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/10.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/11.pth', '/content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/12.pth']
Loading model /content/drive/MyDrive/SKT_FLY_AI /4주차 파이토치/catanddog/00.

In [ ]:
plt.plot(train_acc_hist)
plt.plot(val_acc_hist)
plt.show()